<a href="https://colab.research.google.com/github/atomicguy/stereocards/blob/main/stereocard_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup the Environment

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2782, done.
remote: Counting objects: 100% (2782/2782), done.
remote: Compressing objects: 100% (2318/2318), done.
remote: Total 2782 (delta 711), reused 1299 (delta 429), pack-reused 0
Receiving objects: 100% (2782/2782), 32.76 MiB | 18.74 MiB/s, done.
Resolving deltas: 100% (711/711), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1658497 sha256=942e5c25bbdcebe63d320e7c5622d918261da14820f2b4093aadd3a5d03a4796
  Stored in directory: /tmp/pip-ephem-wheel-cache-_rd68aet/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.2-cp37-none-any.whl size=44011 sha256=e80dfb18410c524ee4bf182702e8e942ed40a6016f841820b84e558e1a6d77b2
  Stored in directory: /root/.cache/pip/wheels/ee/ee/18/c466221ca6900e3efce2f4ea9c329288808679aecdcb2838d3
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78545 sha256=02fcc6b2b216a28f31ced29dce6747c67b911b8b2193aa86843c523461db9e4f
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=02477f47e1a907cb6ad567c05b78748c1f1627593ec0b2ff9638

ERROR: multiprocess 0.70.12.2 has requirement dill>=0.3.4, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.31.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


# Run the Inference

In [4]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [5]:
import os

#recover our saved model
pipeline_config = '/content/gdrive/MyDrive/tf2/ssd-resnet50-augment/pipeline.config'
#generally you want to put the last ckpt from training in here
model_dir = '/content/gdrive/MyDrive/tf2/ssd-resnet50-augment/checkpoint'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join('/content/gdrive/MyDrive/tf2/ssd-resnet50-augment/checkpoint/ckpt-0'))

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function(experimental_relax_shapes=True)
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [6]:
#map labels for inference decoding
label_map_path = '/content/gdrive/MyDrive/tf2/ssd-resnet50-augment/image-pairs_label_map.pbtxt'
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [7]:
import six

label_id_offset = 1

def filter_boxes(detections, max_boxes=2, min_score_thresh=0.5):
  boxes = detections['detection_boxes'][0].numpy()
  classes = (detections['detection_classes'][0].numpy() + label_id_offset).astype(int)
  scores = detections['detection_scores'][0].numpy()

  filtered_boxes = {}

  for i in range(boxes.shape[0]):
    prediction = {}
    if scores is None or scores[i] > min_score_thresh:
      box = boxes[i].tolist()
      display_str = ''
      if classes[i] in six.viewkeys(category_index):
        class_name = category_index[classes[i]]['name']
      else:
        class_name = 'N/A'
      score = scores[i]
      prediction['x0'] = box[0]
      prediction['y0'] = box[1]
      prediction['x1'] = box[2]
      prediction['y1'] = box[3]
      prediction['class'] = class_name
      prediction['score'] = np.float64(score)
      filtered_boxes[class_name] = prediction
  
  return filtered_boxes


In [8]:
# Copy Images locally
!unzip -o /content/gdrive/MyDrive/images/front_jpegs.zip

Streaming output truncated to the last 5000 lines.
  inflating: front_jpegs/1f835cf0-c560-012f-87b4-58d385a7bc34.jpg  
  inflating: front_jpegs/1ff43980-c560-012f-8bb6-58d385a7bc34.jpg  
  inflating: front_jpegs/206cdef0-c560-012f-541e-58d385a7bc34.jpg  
  inflating: front_jpegs/20d175a0-c560-012f-266f-58d385a7bc34.jpg  
  inflating: front_jpegs/21461680-c560-012f-5a2e-58d385a7bc34.jpg  
  inflating: front_jpegs/21be19d0-c560-012f-6725-58d385a7bc34.jpg  
  inflating: front_jpegs/221e7930-c560-012f-7ec7-58d385a7bc34.jpg  
  inflating: front_jpegs/2290b430-c560-012f-9c2e-58d385a7bc34.jpg  
  inflating: front_jpegs/230bef60-c560-012f-2be5-58d385a7bc34.jpg  
  inflating: front_jpegs/237213d0-c560-012f-e4dc-58d385a7bc34.jpg  
  inflating: front_jpegs/23ed5840-c560-012f-1f39-58d385a7bc34.jpg  
  inflating: front_jpegs/246b7b00-c560-012f-f03b-58d385a7bc34.jpg  
  inflating: front_jpegs/24cb7230-c560-012f-369a-58d385a7bc34.jpg  
  inflating: front_jpegs/25476420-c560-012f-7473-58d385a7bc34.jpg

In [11]:
import glob
import json

from functools import singledispatch
from pathlib import Path
from tqdm import tqdm

test_image_paths = glob.glob('/content/front_jpegs/*.jpg')

for image in tqdm (test_image_paths):
    uuid = Path(image).stem
    result_path = f'/content/gdrive/MyDrive/predictions/{uuid}.json'
    if not os.path.exists(result_path):
        try:
            image_np = load_image_into_numpy_array(image)
        except Exception as ex:
            print(f'{image} failed with {ex}')
        input_tensor = tf.convert_to_tensor(
        np.expand_dims(image_np, 0), dtype=tf.float32)
        try:
            detections, _, _ = detect_fn(input_tensor)
            results = filter_boxes(detections)
            results['uuid'] = uuid

            with open(result_path, 'w') as f:
                json.dump(results, f)
        except Exception as ex:
            print(f'{image} caused by {ex}')


 41%|████      | 17204/41810 [00:26<2:58:32,  2.30it/s]

KeyboardInterrupt: ignored

In [ ]:
!cp -r /content/front_jpegs/ /content/gdrive/MyDrive/images/